In [3]:
NonTermSet = set()  # 非终结符集合
TermSet = set()  # 终结符集合
First = {}  # First集
GramaDict = {}  # 处理过的产生式
Code = []  # 读入的产生式
StartSym = ""  # 开始符号
EndSym = '#'  # 结束符号为“#“
Epsilon = "~"  # 由于没有epsilon符号用“~”代替
"""
(1)如果X是终结符，则FIRST(X)={X}
(2)如果X →ε是一个产生式则ε∈ FIRST(X)
(3)如果X是非终结符，且X →Y1 Y2…… Yk,则
   a)如 Y1 =>ε,则FIRST(Y1 )中的所有符号
都在FIRST(X)中
  b) 如Y1 Y2…… Yi-1=> ε, FIRST（ Yi ）,
中的所有符号都在FIRST（X）中
c) Y1 Y2…… Yk=> ε,则ε ∈ FIRST（X）
"""

def getFirst():
    global NonTermSet, TermSet, First, FirstA
    for X in NonTermSet:
        First[X] = set()  # 初始化非终结符First集为空
    for X in TermSet:
        First[X] = set(X)  # 初始化终结符First集为自己
    Change = True
    while Change:  # 当First集没有更新则算法结束
        Change = False
        for X in NonTermSet:#非终结符
            for Y in GramaDict[X]:#这个非终结符的产生式
                k = 0
                eps = True #要加入eps
                while eps and k < len(Y):
                    if not First[Y[k]] - set(Epsilon) <= First[X]:  # 没有一样的就添加，并且改变标志
                        if Epsilon not in First[Y[k]] and Y[k] in NonTermSet:  # Y1到Yi候选式都有~存在
                            eps = False
                        else:
                            First[X] |= First[Y[k]] - set(Epsilon)
                            Change = True
                    if Epsilon not in First[Y[k]]:
                        eps = False
                    k += 1
                if eps:  # X->~或者Y1到Yk均有~产生式
                    First[X] |= set(Epsilon)          
                            
# 显示格式
def display(show_list):
    for item in show_list:
        print('     %-25s' % item, end='')
    print()

# 读取文法
def readGrammar():
    try:
        file = open('grammar.txt', 'r')
        for line in file:
            line = line.replace('\n', "")
            Code.append(line)
    except IOError as e:
        print(e)
        exit()
    finally:
        file.close()
    return Code

# 初始化
def main():
    global NonTermSet, TermSet, First, StartSym, Code
    Code = readGrammar()
    n = int(len(Code))
    print('产生式个数:', n)
    StartSym = Code[0][0]
    print("开始符号：", StartSym)
    print('产生式：G[', StartSym, ']:')
    for i in range(n):
        X, Y = Code[i].split('->')
        print('     ', Code[i])
        NonTermSet.add(X)
        Y = Y.split('|')
        for Yi in Y:
            TermSet |= set(Yi)
        if X not in GramaDict:
            GramaDict[X] = set()
        GramaDict[X] |= set(Y)  # 生成产生式集
    TermSet -= NonTermSet
    print('非终结符:', NonTermSet)
    print('终结符:', TermSet)
    print('文法字典:',GramaDict)
    getFirst()
    print("FIRST集:")
    for k in NonTermSet:
        print('     FIRST[', k, ']: ', First[k])

main()

产生式个数: 4
开始符号： E
产生式：G[ E ]:
      E->eBaA
      A->a|bAcB
      B->aC|dEd
      C->e|dC
非终结符: {'B', 'A', 'E', 'C'}
终结符: {'a', 'e', 'c', 'd', 'b'}
文法字典: {'E': {'eBaA'}, 'A': {'a', 'bAcB'}, 'B': {'aC', 'dEd'}, 'C': {'e', 'dC'}}
FIRST集:
     FIRST[ B ]:  {'a', 'd'}
     FIRST[ A ]:  {'a', 'b'}
     FIRST[ E ]:  {'e'}
     FIRST[ C ]:  {'d', 'e'}
